In [ ]:
import sys
sys.path.insert(0, sys.path[0].removesuffix('/src/jupyter_nb'))
sys.path.insert(1, sys.path[0] + '/src')
print(sys.path)
import pandas as pd
from src.robo import *
from src.utils import *
from src.calcEMA import *
from src.myenv import *
from src.train import *

tail = regression_times + 50
use_cols = ['open_time', 'close']

estimator = 'lightgbm'

In [ ]:
symbol = 'BTCUSDT'
experiment, model = load_model(symbol, estimator)  # cassification_experiment

In [ ]:
df = get_data(symbol, save_database=False, interval='1h', tail=tail)
df.to_csv('log_after_1st_get_data.csv', sep=';', index=False)
df

In [ ]:
df = calc_RSI(df)
df.to_csv('log_after_1st_calc_RSI.csv', sep=';', index=False)
df

In [ ]:
df.dropna(inplace=True)
df

In [ ]:
numeric_features = ['close', 'rsi']
df, _ = regresstion_times(df, numeric_features, regression_times)
df.to_csv('log_after_1st_regresstion_times.csv', sep=';')
df

In [ ]:
df.info()

In [ ]:
window = 14
close_price = 'close'
count = df.shape[0]
df.iloc[count-1:count]['rsi'] = 0


#print(df[['close', 'rsi']].tail())
# _df = df.tail(window + 10)[[close_price, 'rsi']].copy()
_df = df[[close_price, 'rsi']].tail(window+50).copy()
_count = _df.shape[0]
#print(_df)
_df['change'] = _df[close_price].diff()
_df['gain'] = _df.change.mask(_df.change < 0, 0.0)
_df['loss'] = -_df.change.mask(_df.change > 0, -0.0)
_df['avg_gain'] = rma(_df.gain.to_numpy(), window)
_df['avg_loss'] = rma(_df.loss.to_numpy(), window)

_df['rs'] = _df.avg_gain / _df.avg_loss
_df['rsi'] = 100 - (100 / (1 + _df.rs))
#print(_df)

#print(df.iloc[count-1:count]['rsi'])
#print(_df.iloc[_count-1:_count]['rsi'])

df.update(_df.iloc[_count-1:_count]['rsi'])

df.tail()

In [ ]:
cont = 0

In [ ]:
#while True:

# df = get_data(symbol, save_database=False, interval='1h', tail=tail)
_df = get_klines(symbol, interval='1h', max_date=None, limit=1)
_df.to_csv(f'log_after_{cont}_get_klines.csv', sep=';', index=False)

#df = pd.concat([df, _df]).drop_duplicates(keep='last')
if _df.index.isin(df.index):
    df.update(_df)
else:
    df = pd.concat([df, _df])   
df.to_csv(f'log_after_{cont}_merge.csv', sep=';', index=False)

df = calc_RSI(df, last_one=True)
df.to_csv(f'log_after_{cont}_calc_RSI.csv', sep=';', index=False)

df, _ = regresstion_times(df, numeric_features, regression_times, last_one=True)
df.to_csv(f'log_after_{cont}_regresstion_times.csv', sep=';', index=False)

df[['open_time'] + numeric_features].to_csv('log_experiment_data.log', sep=';', index=False)
df_predict = experiment.predict_model(model, df.tail(1))
df_predict.to_csv(f'log_after_{cont}_df_predict.csv', sep=';', index=False)

operacao = df_predict['prediction_label'].values[0]
if (operacao.startswith('SOBE') or operacao.startswith('CAI')):
    send_message(df_predict)
time.sleep(sleep_refresh)
cont += 1

In [ ]:
import pandas as pd
df1 = pd.DataFrame({'open_time': [1, 55, 32, 234], 'close': [3, 4, 5, 6], 'C': [5, 6, 34, 5], 'D': [7, 8, 54, 66], 'label': ['A', 'A', 'B', 'B']}, index=[0,1,2,3])
df1

In [ ]:
df2 = pd.DataFrame({'open_time': [55], 'close': [77]}, index=[1])
df2

In [ ]:
i = df2.iloc[df2.shape[0]-1:df2.shape[0]].index
i

In [ ]:
df2.index.isin(df1.index)

In [ ]:
_df = get_klines('BTCUSDT', interval='1h', max_date=None, limit=200)
_df

In [ ]:
from pycaret.classification.oop import ClassificationExperiment

ca = ClassificationExperiment()
setup = ca.setup(_df, target='symbol')

In [ ]:
best = setup.compare_models()

best

In [ ]:
setup.pull().index[0]

In [ ]:
import datetime 
datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") 

In [ ]:
import pandas as pd
import datetime
import os

df_resultado_simulacao = pd.DataFrame()
if (os.path.exists('resultado_simulacao.csv')):
    df_resultado_simulacao = pd.read_csv('resultado_simulacao.csv', sep=';')

result_simulado = {}
result_simulado['model_name'] = 'BTCUSDT_lr_SL_2.0_RT_1440_RPL_24_1'
result_simulado['data'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
result_simulado['symbol'] = 'BTCUSDT'
result_simulado['estimator'] = 'lightgbm'
result_simulado['train_size'] = 0.7 
result_simulado['start_train_date'] = '2023-06-01'
result_simulado['start_test_date'] = '2023-08-08'
result_simulado['regression_times'] = 24*30
result_simulado['times_regression_profit_and_loss'] = 24
result_simulado['stop_loss'] = 2.0
result_simulado['fold'] = 3
result_simulado['saldo_inicial'] = 100.00
result_simulado['saldo_final'] = 91.87

print(result_simulado)

df = pd.DataFrame.from_dict([result_simulado])

df_resultado_simulacao = pd.concat([df_resultado_simulacao, df], ignore_index=True)
df_resultado_simulacao.sort_values('saldo_final', inplace=True)

df_resultado_simulacao.to_csv('resultado_simulacao.csv', sep=';', index=False)

In [ ]:
from binance.client import Client
client = Client()
klines = client.get_historical_klines(symbol='BTCUSDT', interval='1h', start_str='2010-01-01', limit=1000)



In [ ]:
import pandas as pd


all_klines_cols = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume',
                   'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']

columns = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'quote_asset_volume', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'number_of_trades', 'ignore', 'close_time']

df_klines = pd.DataFrame(data=klines, columns=all_klines_cols)[columns]
df_klines['open_time'] = pd.to_datetime(df_klines['open_time'], unit='ms')
df_klines['close_time'] = pd.to_datetime(df_klines['close_time'], unit='ms')

In [ ]:
df_klines

In [ ]:
download_data()


In [ ]:
all_data = get_data('BTCUSDT', save_database=False, interval='1h', tail=-1, columns=['open_time', 'close'])
all_data

In [ ]:
import numpy as np

at = all_data.tail(1).open_time.dt.strftime('%Y-%m-%d %H').values[0]


In [ ]:
list_of_elements = ['close','open','high','low','volume']
fix_it='close'

combinations_list = []
if len(list_of_elements) > 0:    
    for i in range(1, len(list_of_elements) + 1):
        a = combinations(list_of_elements, i)
        for s in a:
            res = ''
            for j in s:
                res += f'{j},'
            combinations_list.append(res[0:len(res) - 1])

combinations_list

In [ ]:
from itertools import combinations
import pandas as pd

comb = combinations(range(1, 25 + 1), 15)
cols = ['col_' + str(i) for i in range(1, 15 + 1)]
resultado = pd.DataFrame(list(comb), columns=cols)
resultado

In [ ]:
resultado.query('col_1 in (1,2,3,4) and col_15 in (22,23,24,25)')

In [ ]:
import pandas as pd
results = ['a','b','c','d','d','d','d','d',]

pd.DataFrame(results, columns=['status'])['status'].value_counts()

In [1]:
import pandas as pd
import sys
print(sys.path)
sys.path.insert(0, sys.path[0].replace('/src/jupyter_nb', ''))
print(sys.path)
import src.utils as utils
import src.myenv as myenv
import logging

logging.basicConfig(
    level=logging.DEBUG,  # Set the minimum level to be logged
    format="%(asctime)s [%(levelname)s]: %(message)s",
    handlers=[logging.StreamHandler()])


df = pd.read_csv('../../data/ADAUSDT/ADAUSDT.csv', sep=";", index_col='open_time')
df['open_time'] = pd.to_datetime(df.index, unit='ms')
df

['/home/marcelo/des/mg_crypto_trader/src/jupyter_nb', '/usr/lib64/python310.zip', '/usr/lib64/python3.10', '/usr/lib64/python3.10/lib-dynload', '', '/home/marcelo/des/mg_crypto_trader/.env/lib64/python3.10/site-packages', '/home/marcelo/des/mg_crypto_trader/.env/lib/python3.10/site-packages', '/home/marcelo/des/mg_crypto_trader/.env/lib64/python3.10/site-packages/IPython/extensions', '/home/marcelo/.ipython']
['/home/marcelo/des/mg_crypto_trader', '/home/marcelo/des/mg_crypto_trader/src/jupyter_nb', '/usr/lib64/python310.zip', '/usr/lib64/python3.10', '/usr/lib64/python3.10/lib-dynload', '', '/home/marcelo/des/mg_crypto_trader/.env/lib64/python3.10/site-packages', '/home/marcelo/des/mg_crypto_trader/.env/lib/python3.10/site-packages', '/home/marcelo/des/mg_crypto_trader/.env/lib64/python3.10/site-packages/IPython/extensions', '/home/marcelo/.ipython']


,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore,symbol,open_time
open_time,,,,,,,,,,,,,
1523937600000,0.25551,0.28800,0.25551,0.26664,8143693.23,1523941199999,2.165077e+06,4421,2889823.93,7.671342e+05,0,ADAUSDT,2018-04-17 04:00:00
1523941200000,0.26660,0.27798,0.26010,0.26200,8317923.61,1523944799999,2.235633e+06,5149,3819527.71,1.025247e+06,0,ADAUSDT,2018-04-17 05:00:00
1523944800000,0.26221,0.26396,0.24800,0.25664,8420095.41,1523948399999,2.153964e+06,4566,3636154.31,9.302932e+05,0,ADAUSDT,2018-04-17 06:00:00
1523948400000,0.25662,0.26300,0.25489,0.25698,4686043.91,1523951999999,1.215621e+06,3283,1633201.94,4.234452e+05,0,ADAUSDT,2018-04-17 07:00:00
1523952000000,0.25636,0.25998,0.25229,0.25631,3510038.13,1523955599999,8.960957e+05,2408,1349216.91,3.444861e+05,0,ADAUSDT,2018-04-17 08:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695117600000,0.25610,0.25670,0.25470,0.25560,4382746.80,1695121199999,1.121329e+06,2459,2496722.10,6.389632e+05,0,ADAUSDT,2023-09-19 10:00:00
1695121200000,0.25570,0.25620,0.25520,0.25620,1709052.40,1695124799999,4.371322e+05,1315,874909.60,2.238176e+05,0,ADAUSDT,2023-09-19 11:00:00
1695124800000,0.25620,0.25680,0.25500,0.25550,2604897.90,1695128399999,6.669117e+05,1931,1281741.00,3.282942e+05,0,ADAUSDT,2023-09-19 12:00:00


In [2]:
df.sort_index(inplace=True)
df.index.duplicated().sum()

0

In [3]:
all_data = utils.regression_PnL(
            df,
            label=myenv.label,
            diff_percent=1.0,
            max_regression_profit_and_loss=6,
            drop_na=True,
            drop_calc_cols=True,
            strategy=None)

all_data
            

,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore,symbol,open_time,status
open_time,,,,,,,,,,,,,,
1523937600000,0.25551,0.28800,0.25551,0.26664,8143693.23,1523941199999,2.165077e+06,4421,2889823.93,7.671342e+05,0,ADAUSDT,2018-04-17 04:00:00,CAI_1.0
1523941200000,0.26660,0.27798,0.26010,0.26200,8317923.61,1523944799999,2.235633e+06,5149,3819527.71,1.025247e+06,0,ADAUSDT,2018-04-17 05:00:00,CAI_1.0
1523944800000,0.26221,0.26396,0.24800,0.25664,8420095.41,1523948399999,2.153964e+06,4566,3636154.31,9.302932e+05,0,ADAUSDT,2018-04-17 06:00:00,SOBE_1.0
1523948400000,0.25662,0.26300,0.25489,0.25698,4686043.91,1523951999999,1.215621e+06,3283,1633201.94,4.234452e+05,0,ADAUSDT,2018-04-17 07:00:00,SOBE_1.0
1523952000000,0.25636,0.25998,0.25229,0.25631,3510038.13,1523955599999,8.960957e+05,2408,1349216.91,3.444861e+05,0,ADAUSDT,2018-04-17 08:00:00,SOBE_1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695096000000,0.25460,0.25530,0.25400,0.25470,2751381.40,1695099599999,7.000657e+05,1506,1402349.30,3.568489e+05,0,ADAUSDT,2023-09-19 04:00:00,ESTAVEL
1695099600000,0.25480,0.25510,0.25380,0.25390,1367579.30,1695103199999,3.478885e+05,897,306855.40,7.811282e+04,0,ADAUSDT,2023-09-19 05:00:00,ESTAVEL
1695103200000,0.25400,0.25400,0.25270,0.25290,1424998.40,1695106799999,3.610144e+05,1024,482048.20,1.220988e+05,0,ADAUSDT,2023-09-19 06:00:00,SOBE_1.0


In [4]:
all_data['prediction_label'] = all_data['status']
all_data = utils.simule_trading_crypto(all_data, '2023-06-01', '2023-09-19', 100, 1.0)
all_data

2023-09-19 12:15:23,384 [INFO]: [1][CAI_1.0][2023-06-01 01h] => Compra: 0.3700
2023-09-19 12:15:23,386 [INFO]: [2][CAI_1.0][2023-06-01 02h] => Venda: 0.3600 => Diff: -1.41% ==> PnL: $ 101.41
2023-09-19 12:15:23,388 [INFO]: [10][SOBE_1.0][2023-06-01 10h] => Compra: 0.3600
2023-09-19 12:15:23,390 [INFO]: [12][SOBE_1.0][2023-06-01 12h] => Venda: 0.3600 => Diff: 1.22% ==> PnL: $ 102.65
2023-09-19 12:15:23,392 [INFO]: [20][SOBE_1.0][2023-06-01 20h] => Compra: 0.3700
2023-09-19 12:15:23,393 [INFO]: [20][SOBE_1.0][2023-06-01 20h] => Venda: 0.3700 => Diff: -1.08% ==> PnL: $ 101.54
2023-09-19 12:15:23,396 [INFO]: [21][SOBE_1.0][2023-06-01 21h] => Compra: 0.3700
2023-09-19 12:15:23,397 [INFO]: [21][SOBE_1.0][2023-06-01 21h] => Venda: 0.3700 => Diff: -1.19% ==> PnL: $ 100.33
2023-09-19 12:15:23,398 [INFO]: [22][SOBE_1.0][2023-06-01 22h] => Compra: 0.3700
2023-09-19 12:15:23,399 [INFO]: [22][SOBE_1.0][2023-06-01 22h] => Venda: 0.3700 => Diff: -1.19% ==> PnL: $ 99.14
2023-09-19 12:15:23,400 [INFO]:

2023-09-19 12:15:23,426 [INFO]: [72][SOBE_1.0][2023-06-04 00h] => Compra: 0.3800
2023-09-19 12:15:23,427 [INFO]: [72][SOBE_1.0][2023-06-04 00h] => Venda: 0.3800 => Diff: -1.18% ==> PnL: $ 106.80
2023-09-19 12:15:23,431 [INFO]: [90][CAI_1.0][2023-06-04 18h] => Compra: 0.3800
2023-09-19 12:15:23,435 [INFO]: [98][CAI_1.0][2023-06-05 02h] => Venda: 0.3700 => Diff: -1.34% ==> PnL: $ 108.23
2023-09-19 12:15:23,437 [INFO]: [105][CAI_1.0][2023-06-05 09h] => Compra: 0.3700
2023-09-19 12:15:23,438 [INFO]: [111][CAI_1.0][2023-06-05 15h] => Venda: 0.3600 => Diff: -3.49% ==> PnL: $ 112.00
2023-09-19 12:15:23,441 [INFO]: [121][SOBE_1.0][2023-06-06 01h] => Compra: 0.3500
2023-09-19 12:15:23,443 [INFO]: [129][SOBE_1.0][2023-06-06 09h] => Venda: 0.3500 => Diff: 1.17% ==> PnL: $ 113.31
2023-09-19 12:15:23,444 [INFO]: [130][CAI_1.0][2023-06-06 10h] => Compra: 0.3500
2023-09-19 12:15:23,444 [INFO]: [130][CAI_1.0][2023-06-06 10h] => Venda: 0.3500 => Diff: 1.17% ==> PnL: $ 111.98
2023-09-19 12:15:23,445 [IN

1075.140000000001